In [1]:
import os
from tqdm import tqdm
import pandas as pd
from itertools import chain

import cv2
from PIL import Image

In [2]:
# 경로를 지정해주자.

path = "C:/Users/joou7/Untitled Folder/프로젝트종합설계/최종/dataset"

# train data set

In [3]:
train_data_path = os.path.join(path, 'data')

## 데이터 불러오기

In [4]:
# 경로 내에 파일을 모두 불러옴
file_list = os.listdir(train_data_path)

json_file_name = [
    os.path.splitext(file)[0] for file in file_list if file.endswith('.json')
]
jpg_file_name = [
    os.path.splitext(file)[0] for file in file_list if file.endswith('.jpg')
]

## 파일명이 동일한 json과 jpg 파일을 불러옴

In [5]:
common_values = []

for item in json_file_name:
    if item in jpg_file_name:
        common_values.append(item)

In [6]:
json_file_list = []
jpg_file_list = []

In [7]:
for file in file_list:
    if os.path.splitext(file)[0] in common_values:
        if file.endswith('.json'):
            json_file_list.append(file)

In [8]:
for file in file_list:
    if os.path.splitext(file)[0] in common_values:
        if file.endswith('.jpg'):
            jpg_file_list.append(file)

In [9]:
print('train data set')
print('data 폴더에 있는 json 파일 개수 :', len(json_file_name))
print('data 폴더에 있는 jpg 파일 개수 :', len(jpg_file_name))
print('-' * 30)
print('파일명이 동일한 json 파일 개수 :', len(json_file_list))
print('파일명이 동일한 jpg 파일 개수 :', len(jpg_file_list))

train data set
data 폴더에 있는 json 파일 개수 : 1299
data 폴더에 있는 jpg 파일 개수 : 1299
------------------------------
파일명이 동일한 json 파일 개수 : 1123
파일명이 동일한 jpg 파일 개수 : 1123


짝이 안맞는 파일(176개)는 제외한다.

# test data set

In [10]:
test_data_path = os.path.join(path, 'test_data')

## 데이터 불러오기

In [11]:
file_list = os.listdir(test_data_path)

json_file_name = [
    os.path.splitext(file)[0] for file in file_list if file.endswith('.json')
]
jpg_file_name = [
    os.path.splitext(file)[0] for file in file_list if file.endswith('.jpg')
]

## 파일명이 동일한 json과 jpg 파일을 불러옴

In [12]:
common_values = []

for item in json_file_name:
    if item in jpg_file_name:
        common_values.append(item)

In [13]:
json_file_list = []
jpg_file_list = []

In [14]:
for file in file_list:
    if os.path.splitext(file)[0] in common_values:
        if file.endswith('.json'):
            json_file_list.append(file)

In [15]:
for file in file_list:
    if os.path.splitext(file)[0] in common_values:
        if file.endswith('.jpg'):
            jpg_file_list.append(file)

In [16]:
print('test data set')
print('data 폴더에 있는 json 파일 개수 :', len(json_file_name))
print('data 폴더에 있는 jpg 파일 개수 :', len(jpg_file_name))
print('-' * 30)
print('파일명이 동일한 json 파일 개수 :', len(json_file_list))
print('파일명이 동일한 jpg 파일 개수 :', len(jpg_file_list))

test data set
data 폴더에 있는 json 파일 개수 : 149
data 폴더에 있는 jpg 파일 개수 : 149
------------------------------
파일명이 동일한 json 파일 개수 : 35
파일명이 동일한 jpg 파일 개수 : 35


# all data set

In [9]:
data_path = os.path.join(path, 'all_data')
label_path = os.path.join(path, "all_labels")  # 새 yolo label이 들어갈 폴더
save_img_path = os.path.join(path, "all_images")  # 이미지 파일이 존재하는

## 데이터 불러오기

In [4]:
# 경로 내에 파일을 모두 불러옴
file_list = os.listdir(data_path)

json_file_name = [
    os.path.splitext(file)[0] for file in file_list if file.endswith('.json')
]
jpg_file_name = [
    os.path.splitext(file)[0] for file in file_list if file.endswith('.jpg')
]

## 파일명이 동일한 json과 jpg 파일을 불러옴

In [5]:
common_values = []

for item in json_file_name:
    if item in jpg_file_name:
        common_values.append(item)

In [6]:
json_file_list = []
jpg_file_list = []

In [7]:
for file in file_list:
    if os.path.splitext(file)[0] in common_values:
        if file.endswith('.json'):
            json_file_list.append(file)

In [8]:
for file in file_list:
    if os.path.splitext(file)[0] in common_values:
        if file.endswith('.jpg'):
            jpg_file_list.append(file)

In [9]:
print('all data set')
print('data 폴더에 있는 json 파일 개수 :', len(json_file_name))
print('data 폴더에 있는 jpg 파일 개수 :', len(jpg_file_name))
print('-' * 30)
print('파일명이 동일한 json 파일 개수 :', len(json_file_list))
print('파일명이 동일한 jpg 파일 개수 :', len(jpg_file_list))

all data set
data 폴더에 있는 json 파일 개수 : 1448
data 폴더에 있는 jpg 파일 개수 : 1448
------------------------------
파일명이 동일한 json 파일 개수 : 1315
파일명이 동일한 jpg 파일 개수 : 1315


짝이 없는 파일(133개)는 제외한다.

# 이미지 전처리

- 이미지 사이즈 줄이기
- 적응형 이진화 수행
- 노이즈 제거 및 이미지 저장

In [24]:
def resize_image(image, width=None, height=None):
    if width is None and height is None:
        return image
    elif width is None:
        ratio = height / image.shape[0]
        dimension = (int(image.shape[1] * ratio), height)
    elif height is None:
        ratio = width / image.shape[1]
        dimension = (width, int(image.shape[0] * ratio))
    else:
        dimension = (width, height)
    return cv2.resize(image, dimension, interpolation=cv2.INTER_AREA)

In [25]:
# block_size = 15
# C = 2

# def adaptive_binarize_image(gray_image):
#     try:
#         # 적응형 이진화 수행
#         binary_image = cv2.adaptiveThreshold(gray_image, 255,
#                                              cv2.ADAPTIVE_THRESH_MEAN_C,
#                                              cv2.THRESH_BINARY, block_size, C)
#         return binary_image

#     except Exception as e:
#         print(f"오류 발생: {e}")
#         return None

In [26]:
# def otsu_binarize_image(image):
#     # Otsu의 이진화 적용
#     _, binarized_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
#     #_, binarized_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY|cv2.THRESH_OTSU) # 이미지가 아예 블랙
#     return binarized_image

In [27]:
def binarize_image(image, threshold):
    # 이미지를 이진화합니다.
    retval, thresholded_image = cv2.threshold(resized_image, threshold, 255, cv2.THRESH_BINARY)
    return thresholded_image

In [28]:
# os.chdir(data_path)
# image = cv2.imread('ACKS_DY_00501093_0010_006.jpg', cv2.IMREAD_GRAYSCALE)  # gray-scale로 변환
# #image = cv2.imread('ACKS_DY_00500008_0010_003.jpg', cv2.IMREAD_GRAYSCALE)  # gray-scale로 변환

# if image is None:
#     raise Exception("이미지를 불러올 수 없습니다.")

# # 이미지 크기를 조절
# resized_image = resize_image(image, width=640, height=640)

# # 이미지 이진화
# #binarized_image = adaptive_binarize_image(resized_image)
# #binarized_image = otsu_binarize_image(resized_image)
# binarized_image = binarize_image(resized_image, 85)

# if binarized_image is None:
#         raise Exception("이진화된 이미지를 생성할 수 없습니다.")
        
#     # 불필요한 배경 제거
#     #output_image = remove_background(resized_image, binarized_image)
    
#     # 노이즈 제거 수행
# denoised_image = cv2.fastNlMeansDenoising(binarized_image, None, 30, 7, 21)

# # 이미지를 윈도우에 표시합니다.
# cv2.imshow('threshold : 85 + fastNlMeansDenoising', binarized_image)

# # 사용자가 키보드의 아무 키나 누를 때까지 대기합니다.
# cv2.waitKey(0)

# # 모든 윈도우를 닫습니다.
# cv2.destroyAllWindows()


In [29]:
# import numpy as np

# def remove_background(load_image, thresh):
#     # 윤곽선 찾기
#     contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

#     # 빈 마스크 생성
#     mask = np.zeros_like(load_image)

#     # 가장 큰 윤곽선 찾기
#     if contours:
#         largest_contour = max(contours, key=cv2.contourArea)

#         # 마스크에 윤곽선 그리기
#         cv2.drawContours(mask, [largest_contour], -1, (255), thickness=cv2.FILLED)

#         # 원본 이미지에서 배경 제거
#         result = cv2.bitwise_and(load_image, load_image, mask=mask)
#     else:
#         result = load_image  # 윤곽선이 없을 경우 원본 이미지를 반환

In [30]:
for fil in tqdm(jpg_file_list):
    os.chdir(data_path)
    loaded_image = cv2.imread(fil, cv2.IMREAD_GRAYSCALE)  # gray-scale로 변환

    if loaded_image is None:
        raise Exception("이미지를 불러올 수 없습니다.")

    # 이미지 크기를 조절
    resized_image = resize_image(loaded_image, width=640, height=640)

    # 이미지 이진화
    #binarized_image = adaptive_binarize_image(resized_image)
    #binarized_image = otsu_binarize_image(resized_image)
    binarized_image = binarize_image(resized_image, 110) # threshold : 110

    if binarized_image is None:
         raise Exception("이진화된 이미지를 생성할 수 없습니다.")
        
    # 불필요한 배경 제거
    #output_image = remove_background(resized_image, binarized_image)
    
     # 노이즈 제거 수행
    denoised_image = cv2.fastNlMeansDenoising(binarized_image, None, 30, 7, 21)

    # 경로 설정
    os.chdir(save_img_path)

    #이진화 이미지 저장
    cv2.imwrite(fil, denoised_image)

  1%|          | 13/1315 [00:02<04:45,  4.56it/s]


KeyboardInterrupt: 

# 전체 라벨 리스트 생성 : class_id

json 파일에서 바운딩박스의 label을 추출하고 리스트에 저장

In [11]:
label = []

for fil in tqdm(json_file_list):

    #json 파일 내 데이터 읽어오기
    with open(os.path.join(data_path, fil), "r", encoding='utf-8') as file:
        json_data = json.load(file)

    # 바운딩 박스만 추출
    for k in range(len(json_data["Image_Text_Coord"])):
        Image_Text_Coord = list(
            chain.from_iterable(json_data["Image_Text_Coord"]))

        # json -> dataFrame
        df = pd.json_normalize(Image_Text_Coord)

        for item in df['label']:
            label.append(item)

100%|██████████| 1315/1315 [00:16<00:00, 81.32it/s]


In [12]:
label

['甲',
 '子',
 '西',
 '行',
 '錄',
 '自',
 '壬',
 '子',
 '恩',
 '霈',
 '後',
 '易',
 '名',
 '貤',
 '赠',
 '之',
 '典',
 '卽',
 '是',
 '次',
 '苐',
 '應',
 '行',
 '之',
 '事',
 '而',
 '可',
 '川',
 '翁',
 '旣',
 '沒',
 '楮',
 '谷',
 '又',
 '掣',
 '丧',
 '憂',
 '人',
 '事',
 '变',
 '遷',
 '未',
 '遑',
 '以',
 '此',
 '事',
 '警',
 '欬',
 '扵',
 '輦',
 '轂',
 '之',
 '下',
 '者',
 '十',
 '馀',
 '年',
 '于',
 '兹',
 '矣',
 '至',
 '是',
 '士',
 '論',
 '稱',
 '鬱',
 '门',
 '議',
 '竣',
 '𤼲',
 '敦',
 '以',
 '门',
 '内',
 '一',
 '人',
 '本',
 '一',
 '使',
 '孫',
 '人',
 '偕',
 '与',
 '上',
 '京',
 '秀',
 '榮',
 '備',
 '𢿙',
 '焉',
 '盖',
 '屡',
 '被',
 '要',
 '责',
 '有',
 '终',
 '不',
 '敢',
 '辤',
 '者',
 '五',
 '月',
 '𥘋',
 '二',
 '日',
 '與',
 '聖',
 '章',
 '𤼲',
 '程',
 '時',
 '朴',
 '而',
 '健',
 '亦',
 '以',
 '十',
 '二',
 '日',
 '庭',
 '試',
 '行',
 '同',
 '赴',
 '士',
 '淵',
 '飬',
 '之',
 '二',
 '君',
 '以',
 '屏',
 '山',
 '宣',
 '额',
 '之',
 '㑹',
 '附',
 '行',
 '到',
 '後',
 '坪',
 '見',
 '有',
 '屏',
 '山',
 '下',
 '人',
 '持',
 '列',
 '邑',
 '通',
 '文',
 '而',
 '来',
 '者',
 '取',
 '視',
 '之',
 '畧',
 '曰',
 '自',
 '該',
 '曺',
 '以',
 '结'

In [13]:
# 중복 라벨 제거
label_list = list(set(label))

In [14]:
print('전체 한자 라벨 수 : ', len(label))
print('중복 제거한 한자 라벨 수 : ',len(label_list))

전체 한자 라벨 수 :  5847886
중복 제거한 한자 라벨 수 :  6570


## data.yaml

In [15]:
import yaml

In [16]:
#현재 폴더 경로; 작업 폴더 기준
print(os.getcwd())

C:\Users\joou7\Untitled Folder\프로젝트종합설계\최종


In [17]:
list_file_path='label_list.txt'

with open(list_file_path,  'w', encoding='UTF-8') as file:
    for item in label_list:
        file.write('\''+item+'\''+', ')

In [18]:
label_list

['涌',
 '与',
 '訪',
 '幼',
 '斑',
 '魁',
 '嚇',
 '旙',
 '鵬',
 '秊',
 '𠰥',
 '牕',
 '𦤺',
 '銛',
 '预',
 '痕',
 '蔑',
 '围',
 '位',
 '领',
 '䄅',
 '嶂',
 '稅',
 '铁',
 '凉',
 '濈',
 '漁',
 '嗣',
 '鄕',
 '冀',
 '銖',
 '驂',
 '㐮',
 '驻',
 '弑',
 '欬',
 '编',
 '止',
 '菀',
 '绥',
 '寓',
 '萬',
 '砥',
 '信',
 '芹',
 '𥤮',
 '央',
 '穀',
 '颍',
 '𠫇',
 '幽',
 '韜',
 '壯',
 '霮',
 '捧',
 '差',
 '旨',
 '模',
 '茅',
 '寸',
 '杓',
 '沍',
 '挟',
 '内',
 '恁',
 '䳽',
 '槪',
 '瞬',
 '罷',
 '徂',
 '种',
 '薦',
 '乙',
 '壙',
 '飽',
 '侦',
 '好',
 '粵',
 '省',
 '北',
 '泰',
 '捏',
 '邊',
 '缺',
 '畳',
 '窣',
 '荊',
 '饒',
 '𢿙',
 '賚',
 '幡',
 '龍',
 '滙',
 '捽',
 '俊',
 '𥨊',
 '糞',
 '人',
 '恨',
 '洬',
 '𣢸',
 '滛',
 '蒿',
 '獰',
 '陪',
 '剡',
 '勝',
 '棄',
 '像',
 '悼',
 '閙',
 '娚',
 '勤',
 '甘',
 '𠫊',
 '豋',
 '沒',
 '霍',
 '覊',
 '山',
 '烜',
 '酈',
 '伩',
 '逐',
 '商',
 '聆',
 '絃',
 '舡',
 '矄',
 '菊',
 '巹',
 '徇',
 '箇',
 '逮',
 '吿',
 '櫟',
 '镜',
 '涧',
 '嬉',
 '𨔼',
 '唬',
 '衮',
 '霾',
 '㒷',
 '楷',
 '燹',
 '裳',
 '廡',
 '佾',
 '縱',
 '两',
 '𧝬',
 '𫝉',
 '首',
 '短',
 '頓',
 '樵',
 '深',
 '竊',
 '充',
 '齊',
 '滋',
 '聰',
 '譽',
 '戸',
 '氓',
 '俁'

In [19]:
yaml_data = {
    "train": path + '/train.txt',
    "test": path + '/test.txt',
    "val": path + '/valid.txt',
    "names": label_list,  # 클래스 이름
    "nc": len(label_list),  # 클래스 수
}

In [20]:
path

'C:/Users/joou7/Untitled Folder/프로젝트종합설계/최종/dataset'

In [21]:
root_dir = path

In [22]:
with open(os.path.join(root_dir, "data.yaml"), "w") as f:
    yaml.dump(yaml_data, f, encoding=('CP949'))

# 바운딩 박스

json 파일 : **좌상단 x, 좌상단 y**, bouding box의 W, bounding box의 H  


학습에 사용할 yolo 모델 : **class_id, bounding box 중심점의 x, bounding box 중심점의 y**, bouding box의 W, bounding box의 H 이므로 

-> 모델에 맞게 바운딩 박스 변경

**방법**
1. json 파일의 바운딩 박스만 추출
2. 데이터프레임화
3. bbox 모든 정보가 있는 'bbox' 컬럼을 각각 분리 -> left_x, left_y, width, height
4. x, y의 좌상단을 중심점으로 변경
5. 각각의 바운딩 박스의 라벨과 일치하는 라벨 클래스 컬럼 추가
6. 데이터프레임 형식을 리스트로 변환
7. 바운딩 박스 정규화
8. txt 파일로 저장

In [23]:
def normalize_bounding_box(bbox, image_width, image_height):
    """
    바운딩 박스를 원본 이미지 크기에 대해 정규화한다.
    """
    center_x, center_y, width, height = bbox

    norm_center_x = center_x / image_width
    norm_center_y = center_y / image_height
    norm_width = width / image_width
    norm_height = height / image_height

    return norm_center_x, norm_center_y, norm_width, norm_height

In [24]:
def dataframe_to_bounding_boxes(df):
    """
    데이터프레임을 바운딩 박스 리스트로 변환한다.
    """
    bounding_boxes = []
    for _, row in df.iterrows():
        bounding_boxes.append(
            (row['center_x'], row['center_y'], row['width'], row['height']))
    return bounding_boxes

In [25]:
def save_bounding_boxes_with_classId(df, bounding_boxes, file_path):
    """
    클래스 ID가 있는 데이터프레임을 각 행에 맞는 바운딩 박스와 함께 텍스트 파일로 저장한다.
    """
    with open(file_path, 'w') as f:
        for index, row in df.iterrows():
            class_id = row['class_id']
            bbox = bounding_boxes[index]
            line = f"{class_id} " + ' '.join(map(str, bbox)) + '\n'
            f.write(line)

In [26]:
for fil in tqdm(json_file_list):

    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]

    #json 파일 내 데이터 읽어오기
    with open(os.path.join(data_path, fil), "r", encoding='utf-8') as file:
        json_data = json.load(file)

    # 바운딩 박스만 추출
    for k in range(len(json_data["Image_Text_Coord"])):
        Image_Text_Coord = list(
            chain.from_iterable(json_data["Image_Text_Coord"]))

        # json -> dataFrame
        df = pd.json_normalize(Image_Text_Coord)
        df['class_id'] = ''

        # yolo bbox 형태로 변환
        for i in df.index:
            df.loc[i, 'left_x'] = df['bbox'][i][0]
            df.loc[i, 'left_y'] = df['bbox'][i][1]
            df.loc[i, 'width'] = df['bbox'][i][2]
            df.loc[i, 'height'] = df['bbox'][i][3]

        df['center_x'] = df['left_x'] + (df['width'] / 2)
        df['center_y'] = df['left_y'] + (df['height'] / 2)

        # 라벨 클래스 생성
        for j in df.index:
            if df['label'][j] in label_list:
                #df.loc[j, 'class_id'] = label_list.index(df['label'][j])
                df.loc[j, 'class_id'] = 0

        # 데이터프레임을 바운딩 박스 리스트로 변환
        bounding_boxes = dataframe_to_bounding_boxes(
            df[['center_x', 'center_y', 'width', 'height']])

        # 원본 이미지 불러와 이미지 크기 얻기
        os.chdir(data_path)
        image = Image.open(filename + '.jpg')

        os.chdir(path)
        # 정규화된 바운딩 박스를 그립니다.
        normalized_bounding_boxes = [
            normalize_bounding_box(bbox, image.size[0], image.size[1])
            for bbox in bounding_boxes
        ]

        # txt 파일로 저장
        file_path = os.path.join(label_path, f"{filename}.txt")
        save_bounding_boxes_with_classId(df, normalized_bounding_boxes,
                                         file_path)

100%|██████████| 1315/1315 [51:41<00:00,  2.36s/it] 


# 전처리 검증

랜덤한 이미지를 가져와 바운딩 박스를 그려본다.

In [3]:
import random

In [4]:
def get_random_image_from_folder(folder_path):
    """
    랜덤한 이미지를 가져온다.
    """
    # 해당 폴더에 있는 모든 파일 목록을 가져옵니다.
    all_files = os.listdir(folder_path)

    # 이미지 파일만 필터링합니다 (여기서는 jpg, jpeg, png 파일만 고려합니다).
    image_files = [
        f for f in all_files if f.lower().endswith(('.png', '.jpg', '.jpeg'))
    ]

    # 이미지 파일이 하나도 없을 경우 예외 처리합니다.
    if not image_files:
        raise ValueError("해당 폴더에 이미지 파일이 없습니다.")

    # 랜덤으로 하나의 이미지 파일을 선택합니다.
    random_image = random.choice(image_files)

    # 선택된 이미지 이름를 반환합니다.
    return random_image

In [5]:
def get_corresponding_txt_file(image_path):
    # 이미지 파일의 경로에서 확장자를 제거하고 .txt 확장자를 추가합니다.
    txt_file_path = os.path.splitext(image_path)[0] + '.txt'

    # .txt 파일이 존재하는지 확인합니다.
    if not os.path.exists(txt_file_path):
        raise FileNotFoundError(f"{txt_file_path} 파일을 찾을 수 없습니다.")

    return txt_file_path

In [6]:
def convert_format(data):
    # data 형식: class_id 중심점x 중심점y wdith height
    parts = data.strip().split()
    #     print(parts)
    # (중심점x 중심점y wdith height) 형식으로 변환
    new_format = (float(parts[1]), float(parts[2]), float(parts[3]),
                  float(parts[4]))
    return new_format

In [7]:
def draw_normalized_bounding_boxes(image,
                                   bounding_boxes,
                                   color=(0, 255, 0),
                                   thickness=2):
    """
    이미지에 정규화된 바운딩 박스를 그립니다.
    """
    height, width = image.shape[:2]
    for bbox in bounding_boxes:
        x_center, y_center, w, h = bbox

        # 정규화된 좌표를 절대 좌표로 변환
        x_center *= width
        y_center *= height
        w *= width
        h *= height

        # 경계 상자의 좌상단과 우하단 좌표 계산
        x1 = int(x_center - w / 2)
        y1 = int(y_center - h / 2)
        x2 = int(x_center + w / 2)
        y2 = int(y_center + h / 2)

        ## 경계 상자 그리기
        cv2.rectangle(image, (x1, y1), (x2, y2), color, thickness)

In [47]:
# 랜덤 이미지를 선택합니다.
folder_path = save_img_path
#random_image_path = get_random_image_from_folder(folder_path)
random_image_path = "ACKS_DY_00500857_0020_001.jpg"
print("Randomly selected image:", random_image_path)

Randomly selected image: ACKS_DY_00500857_0020_001.jpg


In [48]:
# 랜덤 이미지에 대응하는 txt 파일 경로를 찾습니다.
os.chdir(label_path)
txt_file_path = get_corresponding_txt_file(random_image_path)
print("Corresponding txt file:", txt_file_path)

Corresponding txt file: ACKS_DY_00500857_0020_001.txt


In [49]:
# txt 파일 내용을 읽어 리스트로 저장합니다.
with open(txt_file_path, 'r', encoding='utf-8') as file:
    txt_content = file.readlines()
    list_content = [line.strip() for line in txt_content]
print("Content of the txt file as a list:")
print(list_content[:3])

Content of the txt file as a list:
['0 0.1687624090006618 0.19800931808555697 0.10258107213765719 0.06565014824227022', '0 0.17207147584381205 0.26069462092333756 0.10125744540039709 0.06395595086827616', '0 0.17207147584381205 0.3212621770436256 0.07743216412971542 0.04701397712833545']


In [50]:
# 내용 변환
converted_content = []
for line in list_content:
    converted_line = convert_format(line)
    converted_content.append(converted_line)

# 변환된 내용을 출력합니다.
print("Converted content:")
print(converted_content[:3])

Converted content:
[(0.1687624090006618, 0.19800931808555697, 0.10258107213765719, 0.06565014824227022), (0.17207147584381205, 0.26069462092333756, 0.10125744540039709, 0.06395595086827616), (0.17207147584381205, 0.3212621770436256, 0.07743216412971542, 0.04701397712833545)]


In [51]:
radom_bounding_boxes = converted_content

In [ ]:
# 이미지 불러오기
os.chdir(save_img_path)
loaded_image = cv2.imread(random_image_path)
if loaded_image is None:
    print("이미지를 불러올 수 없습니다.")
else:
    draw_normalized_bounding_boxes(loaded_image, radom_bounding_boxes)

    # 결과 이미지를 화면에 표시합니다.
    cv2.imshow("loaded Image with Normalized Bounding Boxes, threshold:110", loaded_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()